In [ ]:
#All imports

import yfinance as yf
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sqlite3
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import mplfinance as mpf

In [ ]:
def plot_candle(symbol, interval, period):
    #Candlestick plotter using matplotlib finance
    ticker = yf.Ticker(symbol).history(interval = interval, period = period, prepost = True)
    mpf.plot(ticker, type = 'candle', style = 'charles', ylabel = 'Price')

In [ ]:
def plot_line(symbol, interval, period):
    #LIne graph plotter using matplotlib finance library
    ticker = yf.Ticker(symbol).history(interval = interval, period = period, prepost = True)
    mpf.plot(ticker, type = 'line', style = 'charles', ylabel = 'Price')

In [ ]:
def create_ticker(symbol, interval, period):
    #Returns pandas dataframe with datetime, open, close, high, low, volume etc.
    ticker = yf.Ticker(symbol).history(interval = interval, period = period, prepost = True)
    return ticker

In [ ]:
#test ticker
ticker = create_ticker("AAPL", '1d', '1y')

In [ ]:
train_dates = pd.to_datetime(ticker['Datetime'])
print(train_dates.tail(15))